<a href="https://colab.research.google.com/github/gbiamgaurav/NLP/blob/main/Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake news classifier using Bidirectional LSTM

In [68]:
# Load the drive

from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import files
files.upload()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"gbiamgaurav","key":"6c5e2ca88e0e1332c5c4009c0a01fc76"}'}

In [69]:
# Check the kaggle.json

!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Apr 17 12:59 kaggle.json


In [70]:
# Make a directory

!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

In [71]:
# Set permission

!chmod 600 /root/.kaggle/kaggle.json

In [72]:
# Download the file directly from Kaggle

!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [73]:
# Unzip the zip file

!unzip fake-news.zip

Archive:  fake-news.zip
replace submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submit.csv              
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [74]:
import pandas as pd

In [75]:
# df = pd.read_csv("/content/train.csv", engine='python', error_bad_lines=False)

In [76]:
df = pd.read_csv("/content/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [77]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [78]:
df.shape

(20800, 5)

In [79]:
df = df.dropna()

df.shape

(18285, 5)

In [80]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [81]:
## Get the independent & dependent features

X = df.drop('label', axis=1)
y = df['label']

In [82]:
## Check whether dataset is balanced or not

y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [83]:
import tensorflow as tf

In [84]:
tf.__version__

'2.12.0'

In [85]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

In [86]:
## Vocabulary size

voc_size = 5000

In [87]:
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [88]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
messages.reset_index(inplace=True)

In [90]:
## Data Preprocessing

from nltk.stem.porter import PorterStemmer  ## Stemming purpose
ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):

  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)

  corpus.append(review)

In [91]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [92]:
len(corpus)

18285

## One-hot Representation

In [93]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [94]:
onehot_repr

[[1554, 3501, 4664, 1572, 2172, 3052, 1367, 3819, 3438, 4412],
 [232, 3945, 4790, 7, 1578, 871, 118],
 [1845, 2647, 2246, 4872],
 [825, 3894, 688, 3301, 4325, 4021],
 [1615, 1578, 846, 3633, 2463, 2556, 1578, 124, 2166, 550],
 [2500,
  4480,
  1090,
  3963,
  2520,
  2840,
  2859,
  3584,
  3825,
  3598,
  631,
  3289,
  3397,
  1715,
  118],
 [2440, 2251, 2278, 872, 3782, 1499, 4244, 3632, 3328, 3409, 2810],
 [1904, 632, 1661, 4064, 886, 2846, 2840, 31, 3328, 3409, 2810],
 [2341, 2174, 634, 147, 3688, 1991, 2699, 2833, 2840, 2226],
 [436, 1721, 4027, 3775, 4568, 2303, 1607, 1445],
 [520, 4747, 3376, 3336, 329, 113, 884, 4308, 2926, 1246, 3553],
 [3301, 3492, 2172, 1991, 2840, 886],
 [1640, 588, 1562, 2757, 3570, 519, 4308, 3215, 4297],
 [277, 680, 2503, 4446, 2966, 4967, 3213, 3328, 3409, 2810],
 [2042, 3030, 4802, 1505, 2136, 3328, 3409, 2810],
 [171, 3692, 3053, 876, 1641, 3312, 93, 3229, 303, 2506],
 [4960, 2779, 3945],
 [3258, 95, 4928, 290, 2840, 1425, 639, 118],
 [3006, 1685, 47

## Embedding Representation

In [95]:
sent_length = 20

embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 3819, 3438, 4412],
       [   0,    0,    0, ..., 1578,  871,  118],
       [   0,    0,    0, ..., 2647, 2246, 4872],
       ...,
       [   0,    0,    0, ..., 3328, 3409, 2810],
       [   0,    0,    0, ..., 3761,  463,  741],
       [   0,    0,    0, ..., 2473, 1952, 4291]], dtype=int32)

## Create the Model

In [96]:
## Creating the Model

embedding_vector_features = 40

model = Sequential()

model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))

model.add(Bidirectional(LSTM(100)))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [97]:
import numpy as np

In [98]:
X_final = np.array(embedded_docs)

y_final = np.array(y)

In [99]:
## Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [100]:
## Model Training

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 25, batch_size=32, callbacks=[early_stopping])

Epoch 1/25
383/383 [==============================] - 18s 37ms/step - loss: 0.2661 - accuracy: 0.8812 - val_loss: 0.2062 - val_accuracy: 0.9152
Epoch 2/25
383/383 [==============================] - 5s 12ms/step - loss: 0.1350 - accuracy: 0.9496 - val_loss: 0.2273 - val_accuracy: 0.9009


In [101]:
y_pred = model.predict(X_test)

y_pred

189/189 [==============================] - 1s 3ms/step


array([[0.9821862 ],
       [0.00277147],
       [0.0021525 ],
       ...,
       [0.00118316],
       [0.9032665 ],
       [0.9580584 ]], dtype=float32)

In [102]:
y_pred = np.where(y_pred >= 0.5, 1, 0)

y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [103]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

confusion_matrix(y_test, y_pred)

array([[3191,  228],
       [ 370, 2246]])

In [104]:
print("Accuracy of the model: ", accuracy_score(y_test, y_pred))

Accuracy of the model:  0.9009113504556753
